In [ ]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

from pygama import DataGroup
import pygama.io.lh5 as lh5
from pygama.io.daq_to_raw import daq_to_raw
from pygama.io.raw_to_dsp import raw_to_dsp

dsp_dir = '/global/cfs/cdirs/legend/data/cage/LH5/dsp'
dg = DataGroup('legend/CAGE/processing/cage.json', load=False)

# Timestamp Correction

In [ ]:
def dsp_to_hit_cage(f_dsp, f_hit, dg, n_max=None, verbose=False, t_start=None):
    """
    non-general placeholder for creating a pygama 'hit' file.  uses pandas.
    for every file, apply:
    - energy calibration (peakfit results)
    - timestamp correction
    for a more general dsp_to_hit, maybe each function could be given in terms
    of an 'apply' on a dsp dataframe ...
    TODO: create entry config['rawe'] with list of energy pars to calibrate, as
    in energy_cal.py
    """
    apply_ecal = False # user changes these here for now
    apply_tscorr = True

    rawe = ['trapEmax']
    dsp = h5py.File(f_dsp)
    ts_old = np.array(dsp['ORSIS3302DecoderForEnergy']['dsp']['timestamp'])
    ts_old = ts_old.astype(np.int64)
    # create initial 'hit' DataFrame from dsp data
    #hit_store = lh5.Store()
    #data, n_rows = hit_store.read_object(dg.config['input_table'], f_dsp)
    #df_hit = data.get_dataframe()
    ts_sec = []
    # 2. compute timestamp rollover correction (specific to struck 3302)
    if apply_tscorr:
        clock = 100e6 # 100 MHz
        UINT_MAX = 4294967295 # (0xffffffff)
        t_max = UINT_MAX / clock
        #ts = df_hit['timestamp'].values / clock
        ts = ts_old/ clock
        tdiff = np.diff(ts)
        tdiff = np.insert(tdiff, 0 , 0)
        #ind = np.arange(len(tdiff))
        #plt.plot(ind,tdiff)
        iwrap = np.where(tdiff < 0)
        iloop = np.append(iwrap[0], len(ts))
        ts_new, t_roll = [], 0
        for i, idx in enumerate(iloop):
            ilo = 0 if i==0 else iwrap[0][i-1]
            ihi = idx
            ts_block = ts[ilo:ihi]
            ts_block = (np.array(ts_block)).astype(np.uint64)
            #new_block = [t+t_roll for t in ts_block]
            #print(ts_block[0], ts_block[-1], t_roll)
            #print(new_block[0], new_block[-1], t_roll)
            #print(new_block[-1], type(new_block[0]))
            ts_new.append(ts_block + t_roll)
            t_last = ts[ilo-1]
            #print(t_last)
            #t_last = new_block[-1]
            #print(t_last)
            t_diff = t_max - t_last
            t_roll += t_last + t_diff
        ts_sec = np.concatenate(ts_new)
    return ts_sec